In [2]:
import MySQLdb
import _mysql
import numpy as np
import csv
import pandas as pd

In [3]:
## Connect to DB
db=_mysql.connect(host="christest.c5s5wgavsyu2.us-west-2.rds.amazonaws.com",user="christest",
                  passwd="gags1313",db="chrisdb")

db.query("""SELECT column_name FROM information_schema.columns WHERE table_name = 'web_experiments_participant'""")
r=db.store_result() # sends the whole results
header = r.fetch_row(0)
header

(('id',),
 ('MID',),
 ('start_date',),
 ('start_time',),
 ('completion_code',),
 ('completed',),
 ('progress',),
 ('total_tasks',),
 ('bandit_schedule',),
 ('task_order',))

## Get All Subjects that Did Horizon Gain Task

In [4]:
## Get All  Participants 

db.query("""SELECT column_name FROM information_schema.columns WHERE table_name = 'web_experiments_participant'""")
r=db.store_result() # sends the whole results
header = r.fetch_row(0)

db.query("""SELECT * FROM web_experiments_participant""")
r=db.store_result() # sends the whole results
rows = r.fetch_row(1000)


## Search for those with Horizon Gain Task ## 

## Write to CSV 
with open('data_raw/participant_table.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow([x[0] for x in header])

    MIDS = np.array([])
    for row in rows:
        if "bandit" in row[-1]:
            writer.writerow([x for x in row])
            MIDS = np.append(MIDS,row[1])
MIDS

array([], dtype=float64)

In [19]:
df = pd.read_csv('data_raw/participant_table.csv')
df.task_order[0]

"'MASQ', 'horizon_gain', 'horizon_loss'"

## Download Each Subject

In [6]:

## Get Header Names
db.query("""SELECT column_name FROM information_schema.columns WHERE table_name = 'web_experiments_horizon_trial'""")
r=db.store_result() # sends the whole results
header = r.fetch_row(0)

## Get Trials for Each Participant 
for MID in MIDS:
    db.query("""SELECT * FROM web_experiments_horizon_trial WHERE MID = '"""+str(MID)+"""' """)
    r=db.store_result() # sends the whole results
    rows = r.fetch_row(100)
    
    ## Write to CSV 
    with open('data_raw/'+MID+'.csv', 'w') as csvfile:
        writer = csv.writer(csvfile, delimiter=',',quotechar='"', quoting=csv.QUOTE_MINIMAL)
        writer.writerow([x[0] for x in header])
        for row in rows:

            writer.writerow([x for x in row])


## Load Example Subject

In [8]:

df = pd.read_csv('data_raw/'+MID+'.csv')
df.head()

,id,MID,loss_or_reward,game_number,game_trial_number,start_date,trialstart,left_value,right_value,choice,fixed,reaction_time,totalpoints
0,1,12,NaN,0,0,2016-03-04 06:59:13,1457074752370,43,34,0,0,789,43
1,2,12,NaN,0,1,2016-03-04 06:59:15,1457074753163,39,57,1,1,1699,100
2,3,12,NaN,0,2,2016-03-04 06:59:15,1457074754865,44,41,1,1,542,141
3,4,12,NaN,0,3,2016-03-04 06:59:16,1457074755409,40,54,0,0,790,181
4,5,12,NaN,0,4,2016-03-04 06:59:17,1457074756201,33,48,1,2,617,229


In [52]:
df.loc[0,'start_date']


'2016-03-04 06:59:13'

In [9]:
## get date-time 
from dateutil.parser import parse
from datetime import datetime

dt = parse(df.loc[0,'start_date'])
print(dt)
dt.day

2016-03-04 06:59:13


4